ARIMA MODEL:

- p: Number of autoregressive (AR) terms. It represents the lag of the dependent variable.

- d: Number of differencing steps needed to make the data stationary.

- q: Number of moving average (MA) terms. It represents the lag of the forecast error.

# **INSTALL LIBS**

In [1]:
!pip install pprintpp --upgrade
!pip install pmdarima
!pip install statsmodels --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.7 MB/s eta 0:00:00


# **IMPORT LIBRARIES**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pprint import pprint
import statsmodels
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import plotly.graph_objects as go
from scipy import stats
import pmdarima.arima
from pmdarima.arima import auto_arima

# **GET THE DATA**

In [3]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [4]:
train = pd.DataFrame(train, index = range(len(train)))
test = pd.DataFrame(test, index = range(len(test)))

In [5]:
print(f'Train: {len(train)}')
print(f'Test: {len(test)}')
print(train.tail())
print(test.tail())
# 2022-07-01
# 2021-01-01

Train: 901561
Test: 74682
        ProductID        Date    Zip  Units  Revenue        COGS
901556       2213  2013-08-12  15135      1  1070.37  749.794185
901557       2213  2011-12-04  80120      1  1070.37  749.794185
901558       2213  2011-05-29  30028      1  1070.37  749.794185
901559       2213  2014-04-01  65279      1  1070.37  749.794185
901560       2213  2012-12-22  25526      1  1070.37  749.794185
       ProductID        Date    Zip  Units  Revenue         COGS
74677        794  2021-04-09  80602      1  1070.37  1092.312585
74678        794  2021-02-13  80915      1  1070.37  1092.312585
74679        793  2021-08-17  16038      1  1070.37   680.113098
74680        793  2021-02-13  80915      1  1070.37   680.113098
74681        793  2021-04-09  80602      1  1070.37   680.113098


In [6]:
train = train.groupby('Date')[['Revenue', 'Units', 'COGS']].sum().reset_index()
test = test.groupby('Date')[['Revenue', 'Units', 'COGS']].sum().reset_index()

train.columns = ['Date', 'Revenue', 'Units', 'COGS']
test.columns = ['Date', 'Revenue', 'Units', 'COGS']
fig = plt.figure(figsize = (20, 10))
print(train.head())
print(test.head())

         Date     Revenue  Units          COGS
0  2010-07-04  1765391.67    252  1.340364e+06
1  2010-07-05  1425986.10    208  1.112580e+06
2  2010-07-06   302463.00     33  2.373014e+05
3  2010-07-07  1047787.65    181  8.425688e+05
4  2010-07-08   771811.74    121  6.028101e+05
         Date    Revenue  Units           COGS
0  2021-01-01  189558.81     26  153935.401914
1  2021-01-02  280406.70     46  253932.619059
2  2021-01-03  317656.08     48  270310.301163
3  2021-01-04  373463.37     40  316518.998544
4  2021-01-05   31120.11      3   22496.434758


<Figure size 2000x1000 with 0 Axes>

In [7]:
train = train.sort_values(by = 'Date', ascending = True)
test = test.sort_values(by = 'Date', ascending = True)
print(train.tail())
print(test.head())
print(test.iloc[0])

            Date     Revenue  Units          COGS
3713  2020-12-27  1091568.87    214  9.172869e+05
3714  2020-12-28   998180.19    166  8.634828e+05
3715  2020-12-29  1309889.70    196  1.100255e+06
3716  2020-12-30  2194138.80    321  1.828705e+06
3717  2020-12-31    16694.37      1  1.264098e+04
         Date    Revenue  Units           COGS
0  2021-01-01  189558.81     26  153935.401914
1  2021-01-02  280406.70     46  253932.619059
2  2021-01-03  317656.08     48  270310.301163
3  2021-01-04  373463.37     40  316518.998544
4  2021-01-05   31120.11      3   22496.434758
Date          2021-01-01
Revenue        189558.81
Units                 26
COGS       153935.401914
Name: 0, dtype: object


In [8]:
print(f"Train: {len(train)}")
print(f"Test: {len(test)}")

Train: 3718
Test: 535


In [ ]:
max_revenue = max(train['Revenue'])
min_revenue = min(train['Revenue'])

train['Scaled_Revenue'] = (train['Revenue'] - min_revenue) / (max_revenue - min_revenue)
print(train['Scaled_Revenue'].head())
plt.plot(train['Date'], train['Scaled_Revenue'].diff())

0    0.100984
1    0.081557
2    0.017249
3    0.059910
4    0.044114
Name: Scaled_Revenue, dtype: float64


In [ ]:
# ADFULLER TEST
result = adfuller(train['Scaled_Revenue'].diff().dropna())

adf_statistic = result[0]
p_value = result[1]
used_lag = result[2]
n_obs = result[3]
critical_values = result[4]

print('ADF Statistic:', adf_statistic)
print('p-value:', '%.20f'% p_value)
print('Number of Lags Used:', used_lag)
print('Number of Observations Used:', n_obs)
print('Critical Values:')
for key, value in critical_values.items():
    print(f'   {key}: {value}')
print(max(train['Revenue']))
print(min(train['Revenue']))

In [ ]:
model = auto_arima(train['Scaled_Revenue'],
                    exog = train[['Units', 'COGS']],
                  start_p=0, max_p=4,
                  start_q=0, max_q=4,
                  start_P=0,
                  m=12, seasonal=True,
                  d=0, max_d=1,
                  D=0, max_D=2,
                  trace=True,
                  error_action='ignore',
                  suppress_warnings=True,
                  information_criterion='aic',
                  stepwise=True,
                  test = 'kpss',
                  score='mape',
                   )

print(model.aic())

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,1)[12] intercept   : AIC=-8442.443, Time=8.75 sec


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=-8400.578, Time=0.61 sec


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



...

 ARIMA(0,0,1)(1,0,2)[12] intercept   : AIC=-8759.194, Time=37.62 sec


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



 ARIMA(2,0,1)(1,0,2)[12] intercept   : AIC=-8759.897, Time=45.43 sec


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



 ARIMA(1,0,0)(1,0,2)[12]             : AIC=inf, Time=12.22 sec

Best model:  ARIMA(1,0,0)(1,0,2)[12] intercept
Total fit time: 606.243 seconds
-8768.127017525017


In [72]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

sarima_model = SARIMAX(train['Scaled_Revenue'],
                exog = train[['Units', 'COGS']],
                order = (1, 0, 0),      # 2, 1, 1
                seasonal_order =(1, 0, 2, 12))

sarima_result = sarima_model.fit()
print(sarima_result.summary())

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



                                        SARIMAX Results                                        
Dep. Variable:                          Scaled_Revenue   No. Observations:                 3718
Model:             SARIMAX(1, 0, 0)x(1, 0, [1, 2], 12)   Log Likelihood               18572.728
Date:                                 Mon, 24 Feb 2025   AIC                         -37131.456
Time:                                         15:16:21   BIC                         -37087.910
Sample:                                              0   HQIC                        -37115.963
                                                - 3718                                         
Covariance Type:                                   opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Units       8.674e-06   1.71e-07     50.870      0.000    8.34e-06    9.01

In [73]:
start_date = '2020-01-04'
end_date = '2021-07-01'
exog_predict = train[(train['Date'] >= start_date) & (train['Date'] <= end_date)]
exog_predict2 = test[(test['Date'] >= start_date) & (test['Date'] <= end_date)]
# merge = pd.merge(exog_predict, exog_predict2, on = ['Date', 'Revenue', 'Units', 'COGS'])
merge = pd.concat([exog_predict, exog_predict2], axis=0)

print(len(merge))
print(merge.head())
print(merge.tail())

535
            Date    Revenue  Units           COGS  Scaled_Revenue
3361  2020-01-04  278598.60     40  254098.281402        0.015883
3362  2020-01-05  277702.74     46  234413.880309        0.015832
3363  2020-01-06  388538.01     53  349731.438021        0.022176
3364  2020-01-07  350247.87     40  289803.474828        0.019984
3365  2020-01-08  123566.94     12  113248.186317        0.007010
           Date     Revenue  Units          COGS  Scaled_Revenue
173  2021-06-27  1485477.63    209  1.268182e+06             NaN
174  2021-06-28  1796893.56    243  1.548310e+06             NaN
175  2021-06-29  2757337.38    369  2.332883e+06             NaN
176  2021-06-30  4414445.28    584  3.718496e+06             NaN
177  2021-07-01  1619994.60    165  1.359793e+06             NaN


In [74]:
forecast_steps = len(test)
forecast = sarima_result.forecast(len(test), exog = merge[['Units', 'COGS']])
forecast = forecast * (max(train['Revenue']) - min(train['Revenue'])) + min(train['Revenue']) # reverse scaling

def plot_forecasts(forecasts: list[float], title: str) -> None:
    """Function to plot the forecasts."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=test['Date'], y=test['Revenue'], name='Test'))
    fig.add_trace(go.Scatter(x=test['Date'], y = forecast, name= 'Forecast'))

    fig.update_layout(template = "simple_white", font = dict(size=18), title_text=title,
                      width=650, title_x=0.5, height=400, xaxis_title='Date',
                      yaxis_title='Revenue')
    fig.update_layout(
        width=1200,   # Set the width in pixels
        height=600   # Set the height in pixels
    )
    return fig.show()


# Plot the forecasts
plot_forecasts(forecast, 'SARIMAX')

In [75]:
print(len(forecast))
def calculate_mape(actual, forecasted):
    # Ensure both arrays have the same length
    if len(actual) != len(forecasted):
        raise ValueError("Actual and forecasted arrays must have the same length.")

    # Convert to pandas Series to handle NaN and filtering easily
    actual_series = pd.Series(actual)
    forecasted_series = pd.Series(forecasted)

    # Avoid division by zero: filter out zero actual values
    non_zero_mask = actual_series != 0

    # Check if there are any non-zero actual values
    if non_zero_mask.sum() == 0:
        return np.nan  # Return nan if all actual values are zero

    # Calculate MAPE
    mape = np.mean(np.abs((actual_series[non_zero_mask] - forecasted_series[non_zero_mask]) / actual_series[non_zero_mask]))

    return mape

535


In [76]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

actual_values = test['Revenue']
forecast_mean = forecast

mae = mean_absolute_error(actual_values, forecast_mean)
rmse = np.sqrt(mean_squared_error(actual_values, forecast_mean))
r2 = r2_score(actual_values, forecast_mean)

# mape = np.mean(np.abs((actual_values - forecast_mean) / actual_values))
mape = calculate_mape(list(actual_values), list(forecast_mean))

# Print the performance metrics
# print(f"Mean Absolute Error (MAE): {mae}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")
print(f"MAPE: {mape}")

RMSE: 981791.3284103867
R²: 0.04447204708947827
MAPE: 0.5738760631236927
